# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - C
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [2]:
a = rand(10^7); # 1D vector of random numbers, uniform on [0,1)

In [3]:
sum(a)   

5.001814258411753e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

Julia has a `BenchmarkTools.jl` package for easy and accurate benchmarking:

In [6]:
#Pkg.add("BenchmarkTools")
#Pkg.update()

In [7]:
using BenchmarkTools

INFO: Precompiling module BenchmarkTools.


#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [8]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file

# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [9]:
c_sum(a)

5.001814258411629e6

In [10]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

In [11]:
≈  # alias for the `isapprox` function

isapprox (generic function with 3 methods)

In [12]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=sqrt(eps), atol::Real=0, nans::Bool=false, norm::Function)
```

Inexact equality comparison: `true` if `norm(x-y) <= atol + rtol*max(norm(x), norm(y))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, `rtol` defaults to `sqrt(eps(typeof(real(x-y))))`. This corresponds to requiring equality of about half of the significand digits. For other types, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to `vecnorm` but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true
```


We can now benchmark the C code directly from Julia:

In [13]:
c_bench = @benchmark c_sum($a) 

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.296 ms (0.00% GC)
  median time:      13.314 ms (0.00% GC)
  mean time:        14.306 ms (0.00% GC)
  maximum time:     49.988 ms (0.00% GC)
  --------------
  samples:          350
  evals/sample:     1

In [14]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

C: Fastest time was 12.29629 msec


In [15]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 1 entry:
  "C" => 12.2963

# 2. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [17]:
#Pkg.add("PyCall")

In [16]:
using PyCall

INFO: Recompiling stale cache file /Users/aristotle/.julia/lib/v0.6/PyCall.ji for module PyCall.


In [18]:
# Call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [19]:
pysum(a)

5.001814258411629e6

In [20]:
pysum(a) ≈ sum(a)

true

In [21]:
py_list_bench = @benchmark $pysum($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     57.048 ms (0.00% GC)
  median time:      60.212 ms (0.00% GC)
  mean time:        61.727 ms (0.00% GC)
  maximum time:     76.535 ms (0.00% GC)
  --------------
  samples:          81
  evals/sample:     1

In [22]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "C"               => 12.2963
  "Python built-in" => 57.0482

# 3. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [23]:
using Conda 
#Conda.add("numpy")

In [24]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

BenchmarkTools.Trial: 
  memory estimate:  720 bytes
  allocs estimate:  22
  --------------
  minimum time:     4.111 ms (0.00% GC)
  median time:      4.359 ms (0.00% GC)
  mean time:        4.864 ms (0.00% GC)
  maximum time:     21.862 ms (0.00% GC)
  --------------
  samples:          1026
  evals/sample:     1

In [25]:
numpy_sum(apy_list) # python thing

5.001814258411751e6

In [26]:
numpy_sum(apy_list) ≈ sum(a)

true

In [27]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any,Any} with 3 entries:
  "C"               => 12.2963
  "Python numpy"    => 4.11082
  "Python built-in" => 57.0482

# 4. Python, hand-written 

In [28]:
py"""
def py_sum(a):
    s = 0.0
    for x in a:
        s += x
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x13c12a2a8>

In [29]:
py_hand = @benchmark $sum_py($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     484.825 ms (0.00% GC)
  median time:      543.438 ms (0.00% GC)
  mean time:        555.666 ms (0.00% GC)
  maximum time:     720.337 ms (0.00% GC)
  --------------
  samples:          9
  evals/sample:     1

In [30]:
sum_py(apy_list)

5.001814258411629e6

In [31]:
sum_py(apy_list) ≈ sum(a)

true

In [32]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "C"                   => 12.2963
  "Python numpy"        => 4.11082
  "Python hand-written" => 484.825
  "Python built-in"     => 57.0482

# 5. Julia (built-in) 

## Written directly in Julia, not in C!

In [33]:
@which sum(a)

sum(a) in Base at reduce.jl:359

In [34]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.144 ms (0.00% GC)
  median time:      4.853 ms (0.00% GC)
  mean time:        5.026 ms (0.00% GC)
  maximum time:     17.441 ms (0.00% GC)
  --------------
  samples:          993
  evals/sample:     1

In [35]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "C"                   => 12.2963
  "Python numpy"        => 4.11082
  "Python hand-written" => 484.825
  "Python built-in"     => 57.0482
  "Julia built-in"      => 4.14446

# 6. Julia (hand-written) 

In [36]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A
        s = s + a
    end
    s
end

mysum (generic function with 1 method)

In [37]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.421 ms (0.00% GC)
  median time:      13.397 ms (0.00% GC)
  mean time:        14.938 ms (0.00% GC)
  maximum time:     35.724 ms (0.00% GC)
  --------------
  samples:          335
  evals/sample:     1

In [38]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "C"                   => 12.2963
  "Python numpy"        => 4.11082
  "Julia hand-written"  => 12.4207
  "Python hand-written" => 484.825
  "Python built-in"     => 57.0482
  "Julia built-in"      => 4.14446

# Summary

In [39]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 20, "."), lpad(round(value, 2), 10, "."))
end

Python numpy..............4.11
Julia built-in............4.14
C.........................12.3
Julia hand-written.......12.42
Python built-in..........57.05
Python hand-written.....484.82
